In [7]:
import pandas as pd
import ast
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from nltk.corpus import stopwords

# get Indonesian stopword 
list_stopwords = set(stopwords.words('indonesian'))

factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
fact = StemmerFactory()
stemmer = fact.create_stemmer()

In [8]:
df = pd.read_csv('./dataset/database_konten_fixed.csv')
df.head()

,id,title,summary,imgUrl,content,author,createdAt,tags,link,viewers,likes
0,61a420e4-0044-11ee-bbbb-bb155eb672e5,Tips Menjaga Kebugaran Lansia yang Perlu Diket...,"“Seiring dengan bertambahnya umur, menjaga keb...",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seiring bertambahnya usia, ...",dr. Rizal Fadli,29 Mei 2023,"[""kesehatan"",""lansia"",""kebugaran""]",https://www.halodoc.com/artikel/tips-menjaga-k...,0,0
1,61b5264f-0044-11ee-ac36-bb155eb672e5,"Jarang Diketahui, Ini 12 Manfaat Daun Seledri ...",“Ada berbagai manfaat seledri untuk kesehatan....,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seledri merupakan sayuran y...",dr. Fadhli Rizal Makarim,30 Mei 2023,"[""kesehatan"",""makanan""]",https://www.halodoc.com/artikel/jarang-diketah...,0,0
2,61c727ae-0044-11ee-bd96-bb155eb672e5,9 Obat Herbal yang Ampuh Atasi Sakit Lutut dan...,“Obat herbal dapat menjadi pilihan alternatif ...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Sakit lutut dan sendi adala...",dr. Rizal Fadli,10 Mei 2023,"[""herbal"",""kesehatan"",""obat""]",https://www.halodoc.com/artikel/9-obat-herbal-...,0,0
3,61dae8d1-0044-11ee-ac0f-bb155eb672e5,5 Gerakan Yoga yang Bisa Dilakukan oleh Pemula,NaN,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,“Yoga adalah olahraga yang terbilang cukup rin...,dr. Rizal Fadli,03 Desember 2021,"[""olahraga"",""yoga"",""pemula""]",https://www.halodoc.com/artikel/5-gerakan-yoga...,0,0
4,61eb98ae-0044-11ee-a45a-bb155eb672e5,Lakukan 5 Gerakan Yoga Ini untuk Meredakan Stres,NaN,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Yoga dapat membantu mengura...",dr. Verury Verona Handayani,18 Desember 2020,"[""yoga"",""olahraga"",""kesehatan mental"",""stres""]",https://www.halodoc.com/artikel/lakukan-5-gera...,0,0


In [ ]:
df

In [29]:
df['Tags'] = df['Tags'].apply(lambda x: ast.literal_eval(x))
df['Tags']

0                 [kesehatan, lansia, kebugaran]
1                           [kesehatan, makanan]
2                      [herbal, kesehatan, obat]
3                       [olahraga, yoga, pemula]
4      [yoga, olahraga, kesehatan mental, stres]
                         ...                    
185                           [cemas, psikologi]
186         [cemas, psikologi, kesehatan mental]
187                            [cemas, olahraga]
188                             [cemas, makanan]
189                             [cemas, makanan]
Name: Tags, Length: 190, dtype: object

In [30]:
same_summary = []
for i in range(df.shape[0]):
    if df.loc[i,'Summary Makanan'] not in same_summary:
        same_summary.append(df.loc[i,'Summary Makanan'])
    else:
        df.loc[i,'Summary Makanan'] = ''

In [31]:
df.to_csv('dataset_konten_fixed.csv',index=False)
df.to_json('database_konten_fixed.json',orient='records')

In [151]:
df['text'] = df['Judul Artikel']+' '+df['Summary Makanan']+' '+df['Body Artikel'] + ' ' + df['Tags']
df.drop(['Judul Artikel','Summary Makanan','Body Artikel','Tags','Gambar Artikel','Link'],axis=1,inplace=True)

In [192]:
df

,text,cosim
ID,,
0,tips menjaga kebugaran lansia seiring bertam...,0.112032
1,jarang manfaat daun seledri kesehatan manfa...,0.014734
2,obat herbal ampuh atasi sakit lutut sendi ob...,0.000000
3,gerakan yoga pemula yoga olahraga terbilang ...,0.000000
4,lakukan gerakan yoga meredakan stres halodo...,0.000000
...,...,...
185,gangguan kepribadian cemas halodoc jakarta ...,0.022351
186,gejala timbul gangguan kecemasan halodoc jak...,0.014027
187,latihan pernapasan meredakan kecemasan halod...,0.000000


In [153]:
def clean_text(x):
    x = x.lower()
    x = x.strip()
    x = re.sub('[^a-z ]','',x)
    #remove stopword pada list token
    x = ' '.join([word for word in x.split(' ') if not word in list_stopwords])
    # x = stopword.remove(x)
    return x.strip()

In [154]:
df.text = df.text.apply(lambda x: clean_text(x))

In [155]:
df.text

ID
0      tips menjaga kebugaran lansia   seiring bertam...
1      jarang  manfaat daun seledri kesehatan   manfa...
2      obat herbal ampuh atasi sakit lutut sendi   ob...
3      gerakan yoga pemula   yoga olahraga terbilang ...
4      lakukan  gerakan yoga meredakan stres   halodo...
                             ...                        
185    gangguan kepribadian cemas   halodoc jakarta  ...
186    gejala timbul gangguan kecemasan   halodoc jak...
187    latihan pernapasan meredakan kecemasan   halod...
188    makanan plant based meredakan kecemasan faktan...
189    alasan kue redakan gangguan kecemasan   halodo...
Name: text, Length: 190, dtype: object

In [198]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorizing pre-processed movie plots using TF-IDF
tfidfvec = TfidfVectorizer()
vectors = tfidfvec.fit_transform((df.text)).toarray()
  
user_input = 'hari ini saya ingin berolahraga kalo bisa ingin yoga'
user_input = clean_text(user_input)
vector_user = tfidfvec.transform([user_input]).toarray()

In [195]:
vector_user

<1x10539 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [199]:
# Finding cosine similarity between vectors
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(vectors,vector_user)
cos_sim

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.44874588],
       [0.36440062],
       [0.52548598],
       [0.43776083],
       [0.23232681],
       [0.29142877],
       [0.24114642],
       [0.21895982],
       [0.29243951],
       [0.36492001],
       [0.29591606],
       [0.38785474],
       [0.2658656 ],
       [0.51930017],
       [0.26976635],
       [0.39926141],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.09948295],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.01956617],
       [0.02150922],
       [0.02312884],
       [0.01759177],
       [0.        ],
       [0.        ],
       [0.   

In [200]:
import numpy as np
df.iloc[np.argmax(cos_sim)]

text     alasan yoga meredakan tingkat stres wajib dico...
cosim                                             0.017706
Name: 5, dtype: object

In [171]:
cos_sim

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.44874588],
       [0.36440062],
       [0.52548598],
       [0.43776083],
       [0.23232681],
       [0.29142877],
       [0.24114642],
       [0.21895982],
       [0.29243951],
       [0.36492001],
       [0.29591606],
       [0.38785474],
       [0.2658656 ],
       [0.51930017],
       [0.26976635],
       [0.39926141],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.09948295],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.01956617],
       [0.02150922],
       [0.02312884],
       [0.01759177],
       [0.        ],
       [0.        ],
       [0.   

In [173]:
df['cosim'] = cos_sim

Int64Index([5, 16, 3, 6, 18, 14, 12, 4, 13, 11, 8, 17, 15, 9, 7, 10], dtype='int64', name='ID')

In [222]:
def recommendations(input_user, cosine_sim = cos_sim):
    user_input = clean_text(input_user)
    vector_user = tfidfvec.transform([user_input]).toarray()
    print(user_input)
    cos_sim = cosine_similarity(vectors,vector_user)
    df['cosim'] = cos_sim
    
    return df.sort_values('cosim',ascending=False).loc[:10]

In [223]:
recommendations(input_user='hari ini saya ingin makan sehat',cosine_sim=cos_sim)

makan sehat


,text,cosim
ID,,
118,membedakan marah sehat sehat halodoc jakarta...,0.236654
34,catat resep makanan sehat hamil wanita nutr...,0.188249
27,ide makanan sehat makan malam keluarga menu ...,0.177388
19,makanan sehat bergizi lansia menjaga kesehat...,0.176594
23,kenali makanan sehat sempurna manfaatnya a...,0.171531
...,...,...
155,takut ditinggalkan gejala gangguan kepribadian...,0.008984
161,cemas bikin insomnia mengatasinya halodoc ja...,0.008808
109,ketahui manfaat bernostalgia kesehatan mental ...,0.008691


In [191]:
tes = pd.read_json('database_konten.json',orient='records')
tes

,ID,Judul Artikel,Summary Makanan,Gambar Artikel,Body Artikel,Tags,Link
0,0,Tips Menjaga Kebugaran Lansia yang Perlu Dike...,"“Seiring dengan bertambahnya umur, menjaga ke...",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seiring bertambahnya usia, ...","[""kesehatan"",""lansia"",""kebugaran""]",https://www.halodoc.com/artikel/tips-menjaga-k...
1,1,"Jarang Diketahui, Ini 12 Manfaat Daun Seledri...",“Ada berbagai manfaat seledri untuk kesehatan...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Seledri merupakan sayuran y...","[""kesehatan"",""makanan""]",https://www.halodoc.com/artikel/jarang-diketah...
2,2,9 Obat Herbal yang Ampuh Atasi Sakit Lutut da...,“Obat herbal dapat menjadi pilihan alternatif...,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,"Halodoc, Jakarta – Sakit lutut dan sendi adala...","[""herbal"",""kesehatan"",""obat""]",https://www.halodoc.com/artikel/9-obat-herbal-...
3,3,5 Gerakan Yoga yang Bisa Dilakukan oleh Pemula,,https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,“Yoga adalah olahraga yang terbilang cukup rin...,"[""olahraga"",""yoga"",""pemula""]",https://www.halodoc.com/artikel/5-gerakan-yoga...
4,4,Lakukan 5 Gerakan Yoga Ini untuk Meredakan St...,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Yoga dapat membantu mengura...","[""yoga"",""olahraga"",""kesehatan mental"",""stres""]",https://www.halodoc.com/artikel/lakukan-5-gera...
...,...,...,...,...,...,...,...
185,185,5 Gangguan Kepribadian dengan Rasa Cemas Berl...,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Gangguan kepribadian adalah...","[""cemas"",""psikologi""]",https://www.halodoc.com/artikel/5-gangguan-kep...
186,186,15 Gejala yang Timbul dari Gangguan Kecemasan,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta - Siapa yang tidak mengenal g...","[""cemas"",""psikologi"",""kesehatan mental""]",https://www.halodoc.com/artikel/15-gejala-yang...
187,187,Latihan Pernapasan yang Bisa Meredakan Kecema...,,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta – Meskipun merupakan perasaan...","[""cemas"",""olahraga""]",https://www.halodoc.com/artikel/latihan-pernap...
188,188,"Makanan Plant Based Bisa Meredakan Kecemasan,...",,https://d1bpj0tv6vfxyp.cloudfront.net/articles...,"Halodoc, Jakarta - Kesehatan mental sama penti...","[""cemas"",""makanan""]",https://www.halodoc.com/artikel/makanan-plant-...


In [289]:
class Recommendation():
    def __init__(self,dataframe,user_input):
        self.tfidfvec = TfidfVectorizer()
        self.dataframe = self.prepare_dataframe(dataframe)
        self.user_input = user_input
        
    def prepare_dataframe(self,dataframe):
        self.dataframe = pd.read_json(dataframe,orient='records')
        self.dataframe['cosim'] = 0
        self.dataframe['Tags'] = self.dataframe['Tags'].apply(lambda x: ' '.join(ast.literal_eval(x)))
        self.dataframe['text'] = self.dataframe['Judul Artikel']+' '+self.dataframe['Summary Makanan']+' '+self.dataframe['Body Artikel'] + ' ' + self.dataframe['Tags']
        self.dataframe.drop(['Judul Artikel','Summary Makanan','Body Artikel','Tags','Gambar Artikel','Link'],axis=1,inplace=True)
        self.dataframe['text'] = self.dataframe['text'].apply(lambda x: clean_text(x))
        return self.dataframe
    
    def clean_text(self,text):
        self.text = text.lower()
        self.text = self.text.strip()
        self.text = re.sub('[^a-z ]','',self.text)
        self.text = ' '.join([word for word in self.text.split(' ') if not word in list_stopwords])
        self.text = self.text.strip()
        return self.text
    
    def recomendations(self):
        self.vector = self.tfidfvec.fit_transform(self.dataframe['text']).toarray()
        self.user_input = self.clean_text(self.user_input)
        self.vector_user = self.tfidfvec.transform([self.user_input]).toarray()
        self.cos_sim = cosine_similarity(self.vector,self.vector_user)
        self.dataframe['cosim'] = self.cos_sim
        if np.max(self.dataframe['cosim'])<=0.001:
            return self.dataframe.sample(10)
        return self.dataframe.sort_values('cosim',ascending=False).loc[:10]

In [290]:
person1 = Recommendation('database_konten.json','p')
person1.recomendations()

,ID,cosim,text
110,110,0.0,manfaat memiliki sikap optimis kesehatan menta...
168,168,0.0,atasi bosan cemas orang tua pandemi tips hal...
116,116,0.0,ketahui penyebab orang marah halodoc jakarta...
32,32,0.0,kaya nutrisi pilihan makanan sehat lansia m...
114,114,0.0,waspada penyebab mudah marah emosi penyebab ...
94,94,0.0,dampak beban kerja kesehatan mental beban ke...
163,163,0.0,cemas mengganggu kesehatan mental halodoc ja...
28,28,0.0,catat menu makanan sehat hamil muda hamil m...
187,187,0.0,latihan pernapasan meredakan kecemasan halod...
184,184,0.0,depresi cemas memicu mimpi buruk halodoc jak...


In [291]:
person2 = Recommendation('database_konten.json','iqwjdpqnwf')
person2.recomendations()

,ID,cosim,text
153,153,0.0,takut ditinggal orang terdekat tanda idap fear...
59,59,0.0,jaga kesehatan mental direkomendasikan menja...
60,60,0.0,manfaat positive thinking kesehatan mental p...
100,100,0.0,alasan gen z terbuka kesehatan mental keterb...
94,94,0.0,dampak beban kerja kesehatan mental beban ke...
17,17,0.0,bahaya jarang membersihkan matras yoga sadar...
99,99,0.0,manfaat menulis jurnal harian kesehatan mental...
25,25,0.0,menu makanan seharihari sehat keluarga salah...
81,81,0.0,mengenal doomscrolling dampaknya kesehatan men...
5,5,0.0,alasan yoga meredakan tingkat stres wajib dico...


In [1]:
from recommendation import Recommendation

In [7]:
person2 = Recommendation('dataset/database_konten_fixed.json','')
person2.recomendations()

8      6232f51e-0044-11ee-a8fe-bb155eb672e5
118    6a17ed19-0044-11ee-8078-bb155eb672e5
177    6ecdcb60-0044-11ee-9743-bb155eb672e5
189    6fbe7329-0044-11ee-89a9-bb155eb672e5
14     6294e1e4-0044-11ee-affc-bb155eb672e5
                       ...                 
71     66a4504f-0044-11ee-a8d4-bb155eb672e5
109    697b9205-0044-11ee-8a48-bb155eb672e5
157    6d4fb147-0044-11ee-9756-bb155eb672e5
47     64f43b6f-0044-11ee-9d1f-bb155eb672e5
152    6caaa840-0044-11ee-a0eb-bb155eb672e5
Name: id, Length: 190, dtype: object